In [1]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import random

from set_constant import train_path,test_path
from my_generator import my_generator, get_apks_and_types, KFCV_index
from gensim.models import Word2Vec
from set_constant import TYPE,TYPE_list,type_map
from set_constant import word2vec_model_path

def read_matrix(TYPE, TYPE_list, type_map, word2vec_model):
 	# 训练集,测试集
	x_train,y_train,train_apk_count = get_apks_and_types(train_path,TYPE,TYPE_list,type_map,word2vec_model)
	x_test,y_test,test_apk_count = get_apks_and_types(test_path,TYPE,TYPE_list,type_map,word2vec_model)
	all_apk_count = train_apk_count + test_apk_count
	seed = random.random()

	random.seed(seed)
	all_x = np.vstack((x_train, x_test))
	random.shuffle(all_x)

	random.seed(seed)
	all_y = np.vstack((y_train, y_test))
	random.shuffle(all_y)
	return all_x, all_y, all_apk_count

def deep_learning(TYPE,TYPE_list,type_map,word2vec_model, all_x, all_y, all_apk_count):
	if KFCV:
		test_count  = int(all_apk_count * test_split)
		x_train, y_train = all_x[test_count:], all_y[test_count:]

		train_count = all_apk_count - test_count
		x_test, y_test = all_x[:test_count], all_y[:test_count]
		
	else:
		val_count = int(all_apk_count * val_split)
		x_val, y_val = all_x[:val_count], all_y[:val_count]

		test_count  = int(all_apk_count * test_split)
		x_test, y_test = all_x[all_apk_count - test_count:all_apk_count], all_y[all_apk_count - test_count:all_apk_count]

		train_count = all_apk_count - val_count - test_count
		x_train, y_train = all_x[val_count: all_apk_count - test_count], all_y[val_count: all_apk_count - test_count]

	#x_train = x_train.astype('float32')
	#x_test = x_test.astype('float32')
	#x_train /= 255
	#x_test /= 255
	#print('x_train shape:', x_train.shape)
	#print('x_test shape:', x_test.shape)

	# 神经网络
	model = Sequential()
	# 卷积
	model.add(layers.Conv2D(filters=filter_count, kernel_size=kernel_size_, activation='relu', input_shape=(L , K, 1)))
	model.summary()
	#池化
	model.add(layers.MaxPooling2D(maxpooling_size))
	model.add(layers.Flatten())
	model.summary()
	# 第一个全连接
	model.add(layers.Dense(units=first_neuron_count, activation='relu'))
	model.summary()
	# 正则化
	model.add(layers.Dropout(dropout))
	# 第二个全连接
	model.add(layers.Dense(units=TYPE, activation='softmax'))
	model.summary()

	model.compile(optimizer=RMSprop(lr=1e-4),
			loss='binary_crossentropy',
				  metrics=['acc'])

	# history = model.fit(x_train, y_train,
						#epochs=epochs_,
						#batch_size=batch_size,
						#validation_split=validation_split_)
	if KFCV:
		for index in KFCV_index(K, train_count):
			history = model.fit_generator(my_generator(x_train, y_train, index[0], batch_size),
			steps_per_epoch=int(index[2] / batch_size),
			validation_data=my_generator(x_train, y_train, index[1], 1),
			validation_steps=int(index[3] / 1),
			epochs=epochs_,verbose=2)
	else:
		history = model.fit_generator(my_generator(x_train, y_train, [[0,train_count]], batch_size),
		steps_per_epoch=int(train_count / batch_size),
		validation_data=my_generator(x_val, y_val, [[0, val_count]], 1),
		validation_steps=int(val_count / 1),
		epochs=epochs_,verbose=2)
	model.save(save_model_path)

	# 根据结果画图
	acc = history.history['acc']
	val_acc = history.history['val_acc']
	loss = history.history['loss']
	val_loss = history.history['val_loss']

	epochs = range(len(acc))

	plt.plot(epochs, acc, 'bo', label='Training acc')
	plt.plot(epochs, val_acc, 'b', label='Validation acc')
	plt.title('Training and validation accuracy')
	plt.legend()

	plt.figure()

	plt.plot(epochs, loss, 'bo', label='Training loss')
	plt.plot(epochs, val_loss, 'b', label='Validation loss')
	plt.title('Training and validation loss')
	plt.legend()
	
	# 测试集
	#test_loss, test_acc = model.evaluate(x_test, y_test)
	test_loss, test_acc = model.evaluate_generator(my_generator(x_test, y_test, [[0, test_count]], batch_size), steps=int(test_count / batch_size))
	print('Testing and accuracy:', test_acc)
	
	plt.show()
	
	print("Having finished fourth stop:deep learning!")

word2vec_model = Word2Vec.load(word2vec_model_path)
all_x, all_y, all_apk_count = read_matrix(TYPE, TYPE_list, type_map, word2vec_model)
print('Finish reading matrixes')

---  There is this folder.  ---
---  There is this folder.  ---
---  There is this folder.  ---
---  There is this folder.  ---
---  There is this folder.  ---
---  There is this folder.  ---
---  There is this folder.  ---
Finish reading matrixes


In [5]:
from set_constant import save_model_path,L,K
from set_constant import filter_count, kernel_size_,first_neuron_count,dropout,epochs_,batch_size,maxpooling_size
from set_constant import test_split, val_split, KFCV, KFCV_K
deep_learning(TYPE,TYPE_list,type_map,word2vec_model, all_x, all_y, all_apk_count)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 2498, 62, 512)     5120      
Total params: 5,120
Trainable params: 5,120
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 2498, 62, 512)     5120      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 208, 5, 512)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 532480)            0         
Total params: 5,120
Trainable params: 5,120
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_3"
______

ResourceExhaustedError:  OOM when allocating tensor with shape[10,512,2498,62] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradient_tape/sequential_3/max_pooling2d_1/MaxPool/MaxPoolGrad (defined at <ipython-input-1-469a00597bf2>:84) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_3166]

Function call stack:
train_function
